# Convert NETCDF files to COG format (GTIF)
A Cloud Optimized GeoTIFF (COG) is a regular GeoTIFF file, aimed at being hosted on a HTTP file server, with an internal organization that enables more efficient workflows on the cloud. It does this by leveraging the ability of clients issuing ​HTTP GET range requests to ask for just the parts of a file they need.

This notebook converst a file and convert it to COG **using gdal-python on a notebook** and also has the instructions to make the conversion **using gdal on command window** .  

# 0. Libraries

In [12]:
import rioxarray as rio
import xarray as xr
from pyproj import CRS
from affine import Affine
import rasterio
import netCDF4
import matplotlib
import glob
from osgeo import gdal
%matplotlib inline

In [13]:
# Uncomment to find out full path
!pwd

/home/jovyan/s3/GTIF/new_data/luftblick


# 1. Create the name of files based on its time
Each band of the NC file corresponds to an acquisition of every 30 minutes, and that time is in UNIX format on its name. Since the final COG files should have the time stamp in dd-mm-yyyy format, this cell generates a list with the correspondent timestamp to be used for the naming of each output COG file.

In [14]:
# OPEN ONE DATASET #################################################################################
filePath = r'2028/gtif_uibk_20220107.nc'

#Opening the netcdf
#xds = xr.open_dataset(r"C:\TGIF\ncFiles\gtif_uibk_20201101.nc")
xds = xr.open_dataset(filePath)
#xds

In [15]:
# The time in this file is in UNIX format and needs to be changed to timestamp
# Creates a list with the dates associated to each band of the netcdf
listOfTime = xds.time
import datetime
dates = []
for i in listOfTime:
    unixToDatetime = datetime.datetime.fromtimestamp(i) # Unix Time
    dates.append(str(unixToDatetime))

#Umcomment to see result
#dates

/tmp/ipykernel_146/1719349285.py:7: DeprecationWarning: an integer is required (got type DataArray).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  unixToDatetime = datetime.datetime.fromtimestamp(i) # Unix Time


In [16]:
# Re-writting the dates to become the format "yyyymmddThh_mm_ssZ_gtif_uibk_ffp_values"
test_list = dates
char = '-'
char2 = ":"
char3 = " "
# Remove character from Strings list
# using loop + replace() + enumerate()
for idx, ele in enumerate(test_list):
        test_list[idx] = ele.replace(char, '')
        
for idx, ele in enumerate(test_list):
        test_list[idx] = ele.replace(char2, '')

for idx, ele in enumerate(test_list):
        test_list[idx] = ele.replace(char3, 'T')

#Umcomment to see result
print(test_list[0:5])

['20220106T180000', '20220106T183000', '20220106T190000', '20220106T193000', '20220106T200000']


In [17]:
# Convert it to a pandas dataframe
import pandas as pd
test_list = [item + 'Z_gtif_uibk_ffp_values' for item in test_list]

#Uncomment to see result
#test_list[0:5]

In [18]:
df = pd.DataFrame(test_list)
df['band']=range(48)
df['band']=df['band']+1

# Uncomment to see the names of files
len(df)
#print('Total of files:', len(df))

48

## 2. Converts netcdf file to TIF -> Reprojects -> Coverts to COG 
- Extracts the **ffp_value** and creates a ffp_value tif for each of the 48 bands
- The orginal gdal script on the command window to convert a NETCDF to TIFF is:
-- **gdal_translate -of GTiff NETCDF:"input.nc":ffp_values -b 1 output.tif -a_srs EPSG:4326 -a_ullr 11.379291 47.268726 11.392094 47.259460**
- The original gdal script on the command window to reproject a TIFF is:
- - **gdalwarp -t_srs EPSG:3857 input.tif output.tif**
- The original gdal script on the command window to convert a TIFF to COG is:
-- **gdal_translate input.tif output.tif -of COG -co COMPRESS=LZW**
- On python API this would be:
-- **ds = gdal.Translate('output.tif', 'input.tif', options="-of GTiff -co COMPRESS=LZW")**


In [ ]:
# Making a loop to aplly this to all bands of the the netcdf file

from osgeo import gdal
#out_path = "/home/jovyan/s3/GTIF/new_data/luftblick/2022"
#Open existing raster ds
#src_ds = gdal.Open("2022/gtif_uibk_20220112.nc")

# Define path name where netcdf is
old_name = filePath
# Name of the varialbe to be extracted
var_name = 'ffp_values'
# Writes the netcdf with selected varialbe in the format to be used/read for the gdal translate
sublayername =  gdal.Open('NETCDF:"{0}":{1}'.format(old_name, var_name ),gdal.GA_ReadOnly)


#Settings for the conversion to TIFF
projection = '-a_srs EPSG:4326 '
output_format = ' -of GTiff'
#Settings for the conversion to COG
compression =  ' -co COMPRESS=LZW'
output_format2 = ' -of COG'


# Making a loop through all the bands that go from 1 to 48 (which is the leng of the dataframe containing their names)

for i in range(1,len(df)):
    
    #Creates the output name where each band/TIF file will be stored
    outname = old_name[0:5] + df.iloc[i][0] + 'Intermediate.tif'
    #print('\n Verifying if outputname is correct. The output name is: ',outname)
    
    # Defines that band (this is one of the settings to convert to TIF)
    band = ' -b '+str(i)
    
    # SAVES NC FILE TO TIFF #################################################################################################
    
    # Performs the GDAL translate to the band 
    translateOptionText = projection + output_format + band +' -a_ullr '  + str(11.379291) + ' ' + str(47.268726) + ' ' + str(11.392094) + ' ' + str(47.259460)
    #print('\n Verifying if the command is correct. The command is: ', translateOptionText) 
    translateOptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(outname,sublayername,options = translateOptions)

    # REPROJECTING TO ESPG: 3857 ############################################################################################
    old_name[0:5]
    inname =  old_name[0:5] + df.iloc[i][0] + 'Intermediate.tif'
    outname = old_name[0:5] + df.iloc[i][0] + 'Reprojected.tif'
    #print("Confirming Output file name is:", outname)
    #print("Confirming Input file name is :", inname)
    ds = gdal.Warp(outname, inname, dstSRS='EPSG:3857')
    del ds
    
    # CONVERTST REPROJECTED TIF TO COG ######################################################################################
    inname2 =  old_name[0:5] + df.iloc[i][0] + 'Reprojected.tif'
    outname2 = old_name[0:5] + df.iloc[i][0] + '.tif'
    print("Finalizing file n:", band)
    #print("Confirming Output file name is:", outname)
    #print("Confirming Input file name is :", inname)
    ds = gdal.Translate(outname2, inname2, options=" -of COG -co COMPRESS=LZW")
    del ds
print('Done!')

Warning 1: No 1D variable is indexed by dimension ffp


Finalizing file n:  -b 1
Finalizing file n:  -b 2
Finalizing file n:  -b 3
Finalizing file n:  -b 4
Finalizing file n:  -b 5
Finalizing file n:  -b 6
Finalizing file n:  -b 7
Finalizing file n:  -b 8
Finalizing file n:  -b 9
Finalizing file n:  -b 10
Finalizing file n:  -b 11
Finalizing file n:  -b 12
Finalizing file n:  -b 13
Finalizing file n:  -b 14
Finalizing file n:  -b 15
Finalizing file n:  -b 16
Finalizing file n:  -b 17
Finalizing file n:  -b 18
Finalizing file n:  -b 19
Finalizing file n:  -b 20
Finalizing file n:  -b 21
Finalizing file n:  -b 22
Finalizing file n:  -b 23
Finalizing file n:  -b 24
Finalizing file n:  -b 25
Finalizing file n:  -b 26
Finalizing file n:  -b 27
Finalizing file n:  -b 28
Finalizing file n:  -b 29
Finalizing file n:  -b 30
Finalizing file n:  -b 31
Finalizing file n:  -b 32
Finalizing file n:  -b 33
Finalizing file n:  -b 34
Finalizing file n:  -b 35
Finalizing file n:  -b 36
Finalizing file n:  -b 37
Finalizing file n:  -b 38
Finalizing file n:  -

## 2.5 Visualization of COG result

In [9]:
# Some bands have no info for exampple: 21, 9, 23, 24, 38, 42
#import matplotlib.pyplot as plt

# CHANGE HERE THE VALUE BETWEEN 1 TO 48 TO VISUALIZAE THE DIFFERENT BANDS
#i = 5
#path to the file
#fn = old_name[0:5] + df.iloc[i][0] + '.tif'
#print(fn)
#ds = gdal.Open(fn)

# Read raster data
#data_array = ds.GetRasterBand(1).ReadAsArray()
#data_array.shape
#plt.figure(figsize=(10, 10))
#plt.imshow(data_array)
#plt.colorbar()

In [59]:
# Confirming that the resulting reprojected is still a TIFF with the 3857 projection

#i = 2
#path to the file
#fn = old_name[0:5] + df.iloc[i][0] + 'Reprojected.tif'
#print(fn)
#ds = gdal.Open(fn)
#print("'ds' type", type(ds))
#print("___________________")
#print("Projection: ", ds.GetProjection())  # get projection
#print("Columns:", ds.RasterXSize)  # number of columns
#print("Rows:", ds.RasterYSize)  # number of rows
#print("Band count:", ds.RasterCount)  # number of bands

In [49]:
# ANOTHER WAY OF DOING THE LAST CONVERSION TO COG

# Writes the netcdf with selected varialbe in the format to be used/read for the gdal translate
#sublayername =  gdal.Open('NETCDF:"{0}":{1}'.format(old_name, var_name ),gdal.GA_ReadOnly)


# Making a loop through all the bands that go from 1 to 48 (which is the leng of the dataframe containing their names)
#Settings for the conversion to COG
#compression =  ' -co COMPRESS=LZW'
#output_format2 = ' -of COG'
#for i in range(1,len(df)):


    #Creates the output name where each TIF file will be stored
    #outname = old_name[0:5] + df.iloc[i][0] + '.tif'
    #inname =  old_name[0:5] + df.iloc[i][0] + 'Reprojected.tif'
    #outname = old_name.replace('gtif_uibk_20201101.nc','') + df.iloc[i][0] + '.tif'
    #print('\n Verifying if outputname is correct. The output name is: ',outname)
    #print('\Verifying input name is correct. Inptu name is:', inname)
    # Performs the GDAL translate to the band 
    #translateOptionText = output_format2 + compression 
    #print('\n Verifying if the command is correct. The command is: ', translateOptionText) 
    #translateOptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    #gdal.Translate(outname,inname,options = translateOptions)